In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_probability as tfp
import os
from sklearn.model_selection import train_test_split

In [15]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

ModuleNotFoundError: No module named 'google.colab'

In [17]:
# Change base path based on GDrive or local run
base_path = "../dataset"
# base_path = "/gdrive/MyDrive/dressipi_recsys2022_mapped/dataset"

original_data = os.path.join(base_path, 'original_data')
processed_data = os.path.join(base_path, 'processed_data')

#Prepare dataset

In [18]:
tfd = tfp.distributions
tfb = tfp.bijectors
tfpl = tfp.layers

In [19]:
items_features = pd.read_csv(os.path.join(processed_data, "simplified_features.csv"))
items_features.head()

,item_id,feature_idx
0,19021,1
1,19021,23
2,19021,16
3,19021,447
4,19021,2


In [20]:
dataset = np.zeros(
    (items_features['item_id'].max()+1, items_features['feature_idx'].max()+1)
    )
dataset[(items_features['item_id'], items_features['feature_idx'])] = 1

dataset

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

#VAE

In [21]:
def get_prior(num_modes, latent_dimension):
    """
    This function should create an instance of a MixtureSameFamily distribution 
    according to the above specification. 
    The function takes the num_modes and latent_dim as arguments, which should 
    be used to define the distribution.
    Your function should then return the distribution instance.
    """
    return tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=[1/num_modes for _ in range(num_modes)]),
        components_distribution=tfd.MultivariateNormalDiag(
            loc=tf.Variable(tf.random.normal(shape=(num_modes, latent_dimension))),
            scale_diag=tfp.util.TransformedVariable(
                initial_value=tf.ones(
                    shape=(num_modes, latent_dimension)
                ),
                bijector=tfb.Softplus()
            )
        )
    )

In [22]:
def get_kl_regularizer(prior_distribution):
    """
    This function should create an instance of the KLDivergenceRegularizer 
    according to the above specification. 
    The function takes the prior_distribution, which should be used to define 
    the distribution.
    Your function should then return the KLDivergenceRegularizer instance.
    """
    return tfpl.KLDivergenceRegularizer(
        distribution_b=prior_distribution,
        weight=1.0,
        test_points_fn=lambda q: q.sample(3),
        test_points_reduce_axis=None
    )

In [23]:
latent_dimension = 128
starting_dimension = 2**9
prior = get_prior(num_modes=2, latent_dimension=latent_dimension)
kl_regularizer = get_kl_regularizer(prior)

inputs = layers.Input((items_features['feature_idx'].max()+1,))

x = inputs 
dimension = starting_dimension

while(dimension != latent_dimension):
  x = layers.Dense(units=dimension, activation='relu')(x)
  x = layers.BatchNormalization()(x)

  dimension = dimension//2

x = layers.Dense(units=tfpl.MultivariateNormalTriL.params_size(latent_dimension))(x)
outputs = tfpl.MultivariateNormalTriL(latent_dimension, activity_regularizer=kl_regularizer)(x)

encoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='encoder')

encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 905)]             0         
                                                                 
 dense_9 (Dense)             (None, 512)               463872    
                                                                 
 batch_normalization_7 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_8 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_11 (Dense)            (None, 8384)              2154

In [24]:
max_width = 512
ending_dimension = items_features['feature_idx'].max()+1

inputs = layers.Input((latent_dimension,))

x = inputs 
dimension = latent_dimension

while(dimension <= max_width):
  x = layers.Dense(units=dimension, activation='relu')(x)
  x = layers.BatchNormalization()(x)

  dimension = dimension*2

x = layers.Dense(units=tfpl.IndependentBernoulli.params_size(ending_dimension))(x)
outputs = tfpl.IndependentBernoulli(ending_dimension)(x)

decoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='decoder')

decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense_12 (Dense)            (None, 128)               16512     
                                                                 
 batch_normalization_9 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dense_13 (Dense)            (None, 256)               33024     
                                                                 
 batch_normalization_10 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dense_14 (Dense)            (None, 512)               1315

In [25]:
vae = tf.keras.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs), name='vae')
vae.summary()

Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 905)]             0         
                                                                 
 dense_9 (Dense)             (None, 512)               463872    
                                                                 
 batch_normalization_7 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 batch_normalization_8 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_11 (Dense)            (None, 8384)              2154688 

In [26]:
vae.compile(
    optimizer='adam', 
    loss=lambda x_true, x_pred : -tf.reduce_mean(x_pred.log_prob(x_true))
    )

#Training

In [27]:
vae.fit(
    dataset,
    dataset, 
    validation_split=0.2, 
    epochs=100,
    callbacks=[
               tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss',
                  patience=5,
                  restore_best_weights=True,
              )
         ]
    )

Epoch 1/100


2022-04-21 18:41:01.564940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


593/593 [==============================] - ETA: 0s - loss: 124.1309

2022-04-21 18:41:28.548693: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


593/593 [==============================] - 31s 49ms/step - loss: 124.1309 - val_loss: 50.2728
Epoch 2/100
593/593 [==============================] - 28s 48ms/step - loss: 39.2607 - val_loss: 41.3635
Epoch 3/100
593/593 [==============================] - 29s 50ms/step - loss: 32.2972 - val_loss: 37.2438
Epoch 4/100
593/593 [==============================] - 29s 50ms/step - loss: 28.4832 - val_loss: 34.8455
Epoch 5/100
593/593 [==============================] - 29s 49ms/step - loss: 25.8748 - val_loss: 32.4137
Epoch 6/100
593/593 [==============================] - 29s 48ms/step - loss: 24.3166 - val_loss: 31.6039
Epoch 7/100
593/593 [==============================] - 29s 49ms/step - loss: 23.2171 - val_loss: 30.7962
Epoch 8/100
593/593 [==============================] - 29s 49ms/step - loss: 22.3839 - val_loss: 30.4323
Epoch 9/100
593/593 [==============================] - 29s 49ms/step - loss: 21.6482 - val_loss: 29.8156
Epoch 10/100
593/593 [==============================] - 29s 49ms/s

In [28]:
np.save(os.path.join(processed_data, "compressed_features"), encoder(dataset).mean())